In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score 
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/sales_data.csv",encoding='latin_1')
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [ ]:
data = data.dropna() # null 값 제거
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    406829 non-null  object 
 1   StockCode    406829 non-null  object 
 2   Description  406829 non-null  object 
 3   Quantity     406829 non-null  int64  
 4   InvoiceDate  406829 non-null  object 
 5   UnitPrice    406829 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      406829 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.9+ MB


In [ ]:
column = data.columns.values.tolist() # 고유값 개수 확인

for i in column:
    print(f"{i} = {data[i].nunique()}")

InvoiceNo = 22190
StockCode = 3684
Description = 3896
Quantity = 436
InvoiceDate = 20460
UnitPrice = 620
CustomerID = 4372
Country = 37


In [ ]:
# 상품 수량이 음수인 경우 제거
print(data[data['Quantity']<=0].shape[0])
data = data[data['Quantity']>0]

8905


In [ ]:
# 상품 가격이 0 이하인 경우를 제거
print(data[data['UnitPrice']<=0].shape[0])
df = data[data['UnitPrice']>0]

40


In [ ]:
# 상품 코드가 숫자로 이루어져 있지 않은 경우 삭제
data['num'] = data['StockCode'].apply(lambda x: any(c.isdigit() for c in x))
data = data[data['num'] == True]
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,num
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,True
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,True
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,True
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,True
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,True
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,True
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,True
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,True
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,True


In [ ]:
# 나라이름 숫자형으로 변환
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()

data['Country'] = label_encoder.fit_transform(data['Country'])
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,num
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,35,True
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,35,True
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,35,True
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,35,True
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,35,True
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,13,True
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,13,True
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,13,True
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,13,True


In [ ]:
# 가장 잘 팔린 상품 찾기
description_quantity = data.groupby('Description')['Quantity'].sum()
description_quantity.sort_values(ascending = False)

Description
PAPER CRAFT , LITTLE BIRDIE           80995
MEDIUM CERAMIC TOP STORAGE JAR        77916
WORLD WAR 2 GLIDERS ASSTD DESIGNS     54415
JUMBO BAG RED RETROSPOT               46181
WHITE HANGING HEART T-LIGHT HOLDER    36725
                                      ...  
VINTAGE BEAD PINK JEWEL STAND             1
CRACKED GLAZE EARRINGS RED                1
JADE CRYSTAL+GLASS BRACELET               1
CRACKED GLAZE EARRINGS BROWN              1
CAKE STAND LACE WHITE                     1
Name: Quantity, Length: 3872, dtype: int64

In [ ]:
print(data['Description'].unique())

['WHITE HANGING HEART T-LIGHT HOLDER' 'WHITE METAL LANTERN'
 'CREAM CUPID HEARTS COAT HANGER' ... 'PINK CRYSTAL SKULL PHONE CHARM'
 'CREAM HANGING HEART T-LIGHT HOLDER' 'PAPER CRAFT , LITTLE BIRDIE']


In [ ]:
#가장 많이 판매되었던 상품을 1, 2, 3, 4까지 라벨링
data['Description'] = data['Description'].replace(['PAPER CRAFT , LITTLE BIRDIE','MEDIUM CERAMIC TOP STORAGE JAR','WORLD WAR 2 GLIDERS ASSTD DESIGNS','JUMBO BAG RED RETROSPOT'],[1,2,3,4])
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,num
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,35,True
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,35,True
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,35,True
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,35,True
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,35,True
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,13,True
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,13,True
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,13,True
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,13,True


In [ ]:
# 상위 4개 상품 이외는 0으로 변환

data.loc[(data['Description'] != 1) & (data['Description'] != 2) & (data['Description'] != 3) & (data['Description'] != 4), 'Description'] = 0
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,num
0,536365,85123A,0,6,12/1/2010 8:26,2.55,17850.0,35,True
1,536365,71053,0,6,12/1/2010 8:26,3.39,17850.0,35,True
2,536365,84406B,0,8,12/1/2010 8:26,2.75,17850.0,35,True
3,536365,84029G,0,6,12/1/2010 8:26,3.39,17850.0,35,True
4,536365,84029E,0,6,12/1/2010 8:26,3.39,17850.0,35,True
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,0,12,12/9/2011 12:50,0.85,12680.0,13,True
541905,581587,22899,0,6,12/9/2011 12:50,2.10,12680.0,13,True
541906,581587,23254,0,4,12/9/2011 12:50,4.15,12680.0,13,True
541907,581587,23255,0,4,12/9/2011 12:50,4.15,12680.0,13,True


In [ ]:
# 필요없는 컬럼 삭제 
df = data.drop('num',axis = 1)
df = df.drop('StockCode',axis = 1)
df = df.drop('InvoiceDate',axis = 1)
df

,InvoiceNo,Description,Quantity,UnitPrice,CustomerID,Country
0,536365,0,6,2.55,17850.0,35
1,536365,0,6,3.39,17850.0,35
2,536365,0,8,2.75,17850.0,35
3,536365,0,6,3.39,17850.0,35
4,536365,0,6,3.39,17850.0,35
...,...,...,...,...,...,...
541904,581587,0,12,0.85,12680.0,13
541905,581587,0,6,2.10,12680.0,13
541906,581587,0,4,4.15,12680.0,13
541907,581587,0,4,4.15,12680.0,13


## 모델링 

In [ ]:
skf_iris = StratifiedKFold(n_splits=3)

cv_accuracy=[]
cv_precision=[]
cv_recall=[]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396503 entries, 0 to 541908
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    396503 non-null  object 
 1   Description  396503 non-null  object 
 2   Quantity     396503 non-null  int64  
 3   UnitPrice    396503 non-null  float64
 4   CustomerID   396503 non-null  float64
 5   Country      396503 non-null  int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 21.2+ MB


In [ ]:
df = df.astype({'Description':'int'}) # type 변환

In [ ]:
cnt_iter = 0
for train_idx, test_idx in skf_iris.split(df, df['Description']): 
    cnt_iter += 1
    label_train = df['Description'].iloc[train_idx]
    label_test = df['Description'].iloc[test_idx]
    print('교차검증 :{}'.format(cnt_iter))
    print('학습 레이블데이터 분포\n', label_train.value_counts())
    print('검증 레이블데이터 분포\n', label_test.value_counts())

교차검증 :1
학습 레이블데이터 분포
 0    262808
4      1079
3       315
2       132
1         1
Name: Description, dtype: int64
검증 레이블데이터 분포
 0    131405
4       539
3       158
2        66
Name: Description, dtype: int64
교차검증 :2
학습 레이블데이터 분포
 0    262809
4      1078
3       316
2       132
Name: Description, dtype: int64
검증 레이블데이터 분포
 0    131404
4       540
3       157
2        66
1         1
Name: Description, dtype: int64
교차검증 :3
학습 레이블데이터 분포
 0    262809
4      1079
3       315
2       132
1         1
Name: Description, dtype: int64
검증 레이블데이터 분포
 0    131404
4       539
3       158
2        66
Name: Description, dtype: int64


/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


In [ ]:
## macro

result_clf = DecisionTreeClassifier(random_state=100)
n_iter = 0 

for train_idx, test_idx in skf_iris.split(df, df['Description']):
    X_train, X_test = df.iloc[train_idx], df.iloc[test_idx]
    
    y_train, y_test = df['Description'].iloc[train_idx], df['Description'].iloc[test_idx]
    
    #학습 진행
    result_clf.fit(X_train, y_train)
    #예측
    pred = result_clf.predict(X_test)
    
    #정확도 측정
    n_iter +=1 
    accuracy = np.round(accuracy_score(y_test, pred),3)
    precision = np.round(precision_score(y_test,pred, average='macro'),3)
    recall = np.round(recall_score(y_test, pred, average='macro'),3)
    print('\n{} 교차검증 정확도 : {}, 교차검증 precision :{}, 교차검증 recall :{}'.format(n_iter, accuracy, precision, recall))
    cv_accuracy.append(accuracy)
    cv_precision.append(precision)
    cv_recall.append(recall)
    
print('\n')
print('\n 평균 검증 정확도', np.mean(cv_accuracy), np.mean(cv_precision), np.mean(cv_recall))

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(



1 교차검증 정확도 : 1.0, 교차검증 precision :1.0, 교차검증 recall :1.0


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



2 교차검증 정확도 : 1.0, 교차검증 precision :0.8, 교차검증 recall :0.8

3 교차검증 정확도 : 1.0, 교차검증 precision :0.8, 교차검증 recall :0.706



 평균 검증 정확도 1.0 0.8666666666666667 0.8353333333333334


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
## micro 

result_clf = DecisionTreeClassifier(random_state=100)
n_iter = 0 

for train_idx, test_idx in skf_iris.split(df, df['Description']):
    X_train, X_test = df.iloc[train_idx], df.iloc[test_idx]
    
    y_train, y_test = df['Description'].iloc[train_idx], df['Description'].iloc[test_idx]
    
    #학습 진행
    result_clf.fit(X_train, y_train)
    #예측
    pred = result_clf.predict(X_test)
    
    #정확도 측정
    n_iter +=1 
    accuracy = np.round(accuracy_score(y_test, pred),3)
    precision = np.round(precision_score(y_test,pred, average='micro'),3)
    recall = np.round(recall_score(y_test, pred, average='micro'),3)
    print('\n{} 교차검증 정확도 : {}, 교차검증 precision :{}, 교차검증 recall :{}'.format(n_iter, accuracy, precision, recall))
    cv_accuracy.append(accuracy)
    cv_precision.append(precision)
    cv_recall.append(recall)
    
print('\n')
print('\n 평균 검증 정확도', np.mean(cv_accuracy), np.mean(cv_precision), np.mean(cv_recall))

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(



1 교차검증 정확도 : 1.0, 교차검증 precision :1.0, 교차검증 recall :1.0

2 교차검증 정확도 : 1.0, 교차검증 precision :1.0, 교차검증 recall :1.0

3 교차검증 정확도 : 1.0, 교차검증 precision :1.0, 교차검증 recall :1.0



 평균 검증 정확도 1.0 0.9333333333333332 0.9176666666666667


In [ ]:
## weighted 

result_clf = DecisionTreeClassifier(random_state=100)
n_iter = 0 

for train_idx, test_idx in skf_iris.split(df, df['Description']):
    X_train, X_test = df.iloc[train_idx], df.iloc[test_idx]
    
    y_train, y_test = df['Description'].iloc[train_idx], df['Description'].iloc[test_idx]
    
    #학습 진행
    result_clf.fit(X_train, y_train)
    #예측
    pred = result_clf.predict(X_test)
    
    #정확도 측정
    n_iter +=1 
    accuracy = np.round(accuracy_score(y_test, pred),3)
    precision = np.round(precision_score(y_test,pred, average='weighted'),3)
    recall = np.round(recall_score(y_test, pred, average='weighted'),3)
    print('\n{} 교차검증 정확도 : {}, 교차검증 precision :{}, 교차검증 recall :{}'.format(n_iter, accuracy, precision, recall))
    cv_accuracy.append(accuracy)
    cv_precision.append(precision)
    cv_recall.append(recall)
    
print('\n')
print('\n 평균 검증 정확도', np.mean(cv_accuracy), np.mean(cv_precision), np.mean(cv_recall))

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(



1 교차검증 정확도 : 1.0, 교차검증 precision :1.0, 교차검증 recall :1.0


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



2 교차검증 정확도 : 1.0, 교차검증 precision :1.0, 교차검증 recall :1.0

3 교차검증 정확도 : 1.0, 교차검증 precision :1.0, 교차검증 recall :1.0



 평균 검증 정확도 1.0 0.9555555555555555 0.9451111111111111


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
